In [ ]:
import datasets

#load the japanese dataset
dataset_ja = datasets.load_dataset("cc100", lang="ja")

In [ ]:
# sample an element
dataset_ja["train"][961563]

In [ ]:
from src.make_datasets import make_sentence_files

make_sentence_files(dataset_ja["train"])

In [4]:
import glob
import cfg
from random import sample
from tqdm import tqdm
import os

num_files = 5

# read text files as a list
jp_sentence_files = glob.glob(cfg.JP_SENTENCES_DIR + "/*.txt")

# sample num_files
sampled_files=sample(jp_sentence_files,5)
print(sampled_files)

#read all the lines as a list
all_lines = []
for filename in sampled_files:
    with open(filename) as f:
        lines = f.read().splitlines()
        
    all_lines.extend(lines)

print("number of lines sampled:", len(all_lines))

#combine into a single file

tempfile = os.path.join(cfg.DATA_DIR, "temp.txt")
with open(tempfile, "w") as f:

            for sentence in tqdm(all_lines):
                
                # remove newlines
                line = sentence.strip()

                # do not save empty items such as
                if sentence != []:

                    f.writelines(sentence + '\n')

['/work/data/jp_sentences/sent_43.txt', '/work/data/jp_sentences/sent_38.txt', '/work/data/jp_sentences/sent_37.txt', '/work/data/jp_sentences/sent_21.txt', '/work/data/jp_sentences/sent_32.txt']
number of lines sampled: 58262277


100%|███████████████████████████████████████████████████████████████████████████████████████████| 58262277/58262277 [05:58<00:00, 162611.27it/s]


In [ ]:
import sentencepiece as spm
import time

#spm.SentencePieceTrainer.train(input=jp_sentence_files, 
#                               model_prefix='m', vocab_size=1000)

vocab_size = 32000

start = time.time()
model_prefix = "cc100_jp" + "_vocab_" + str(vocab_size)
spm.SentencePieceTrainer.train(input=tempfile
                                       , model_prefix=model_prefix
                                       , vocab_size=vocab_size
                                       , character_coverage = 0.9995
                                       , num_threads=60
                                       , train_extremely_large_corpus=True
                                      ) 
print("Trained {} in {} seconds".format(model_prefix, time.time()-start))

In [ ]:
print("test")

In [ ]:
# changing the ja into jp here :)
import cfg
import sentencepiece as spm
import time
#from multiprocessing import Pool


languages = ["jp"]#, "en"]
sentences_fp = {"jp": jp_sentence_files}#, "en": cfg.EN_SENTENCES_FILE}
vocab_sizes = [8000, 16000, 24000, 32000, 48000]
character_coverages = {"jp": 0.995, "en": 1.00}
#multiproc_pool_size = 6
num_threads = 60

def train_single_param(lang, vocab_size):
    """ Train a single parameter combination"""
    
    start = time.time()
    model_prefix = "cc100_"+lang + "_vocab_" + str(vocab_size)
    spm.SentencePieceTrainer.train(input=sentences_fp[lang]
                                       , model_prefix=model_prefix
                                       , vocab_size=vocab_size
                                       , character_coverage = character_coverages[lang]
                                       , num_threads=num_threads
                                       , train_extremely_large_corpus=True
                                      ) 
    print("Trained {} in {} seconds".format(model_prefix, time.time()-start))
    


#loop over parameters for the actual train in multiprocessing pool
param_combinations = [(lang,vocab_size) for lang in languages for vocab_size in vocab_sizes]
#p = Pool(multiproc_pool_size)

#p.starmap(train_single_param, param_combinations)

#for lang, vocab_size in param_combinations:
#    train_single_param(lang, vocab_size)
train_single_param(param_combinations[0][0],param_combinations[0][1])